# Feature-wise analysis (S. 6.2)

Here I examine the conformance of various Ovidian (and non-Ovidian) works to general Ovidian style. It is demonstrated that using the Mahalanobis distance at a 99% confidence level is a fairly reliable indicator of Ovidian vs non-Ovidian authorship, and that none of the _Heroides_ display any statistical reason to reject them in terms of poetic style.

In [5]:
from mqdq import utils, babble, elegy
from mqdq import line_analyzer as la
from mqdq import mahalanobis as maha

import bs4
import glob

import numpy as np
import pandas as pd
import scipy as sp

In [13]:
collection = []

# Several lines need to be manually deleted, because when we make wide vectors
# we treat couplets as a unit (so we must have a matching number of H and P.)
# In some poems, we have corrupt lines, and so we delete the H that matches
# a corrupt P and vice versa.

ep = babble.bookbabs('corpus/OV-epis.xml', name="Ep.")
for b in ep:
    b.author = 'Ovid'
collection.extend(ep)

tr = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-tri*.xml')), name="Tr.")
for b in tr:
    b.author = 'Ovid'
collection.extend(tr)

am = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-amo*.xml')), name="Am.")
for b in am:
    b.author = 'Ovid'
collection.extend(am)

tib = babble.multi_bookbabs(sorted(glob.glob('corpus/TIB-ele*.xml')), name="Tib.")
del tib[1].raw_source[24]
for b in tib:
    b.author = 'Tibullus'
collection.extend(tib)

prop = babble.multi_bookbabs(sorted(glob.glob('corpus/PROP-ele*.xml')), name="Prop.")
for b in prop:
    b.author = 'Propertius'
del prop[55].raw_source[28]
collection.extend(prop)

cat = babble.bookbabs('corpus/CATVLL-carm.xml', name="Cat.")
cat_ele = [x for x in cat if x.elegiac and len(x) > 20]
for b in cat_ele:
    b.author = 'Catullus'
del cat_ele[3].raw_source[46]
collection.extend(cat_ele)

pon = babble.multi_bookbabs(sorted(glob.glob('corpus/OV-pon*.xml')), name="Pont.")
for b in pon:
    b.author = 'Ovid'
del pon[1].raw_source[8]
del pon[7].raw_source[18]
collection.extend(pon)


In [17]:
nux = babble.Babbler.from_file('./corpus/OV-nux.xml', name='Nux')
nux.author = 'ps-Ovid'

In [21]:
collection.append(nux)

In [22]:
trim = [x for x in collection if x.elegiac and len(x) > 20]
vecs_trim = elegy.vectorise_babs(trim)

In [30]:
nux_vec = vecs_trim[vecs_trim.Author == 'ps-Ovid'].drop(['Author','Work','Poem'],axis=1)

In [11]:
vecs_trim.head()

,Author,Work,Poem,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
0,Ovid,Ep.,Ep. 1,0.086207,0.500000,0.500000,0.448276,0.241379,0.706897,0.810345,...,0.0,0.206897,0.068966,0.396552,1.0,0.094828,4.393948,0.739842,116,0.0
1,Ovid,Ep.,Ep. 2,0.189189,0.527027,0.581081,0.391892,0.283784,0.743243,0.878378,...,0.0,0.202703,0.067568,0.337838,1.0,0.114865,4.071062,1.027448,148,0.0
2,Ovid,Ep.,Ep. 3,0.220779,0.493506,0.519481,0.480519,0.181818,0.597403,0.818182,...,0.0,0.116883,0.025974,0.324675,1.0,0.090909,3.845700,0.484285,154,0.0
3,Ovid,Ep.,Ep. 4,0.102273,0.511364,0.545455,0.465909,0.147727,0.659091,0.829545,...,0.0,0.215909,0.045455,0.329545,1.0,0.073864,3.822098,0.893575,176,0.0
4,Ovid,Ep.,Ep. 5,0.215190,0.455696,0.632911,0.417722,0.164557,0.658228,0.911392,...,0.0,0.202532,0.037975,0.341772,1.0,0.056962,3.727347,0.713715,158,0.0


In [24]:
# Take every poem by Ovid and use that to form the comparison distribution.
# The mean of this dataframe is the Ovidian centroid, but the whole frame
# is used to build the covariance matrix.

ovid_dist = vecs_trim[vecs_trim.Author=='Ovid'].drop(['Author','Work','Poem'],axis=1)
ovid_dist

,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,H4CF,H1DI,H2DI,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
0,0.086207,0.500000,0.500000,0.448276,0.241379,0.706897,0.810345,0.551724,0.586207,0.051724,...,0.0,0.206897,0.068966,0.396552,1.000000,0.094828,4.393948,0.739842,116,0.000000
1,0.189189,0.527027,0.581081,0.391892,0.283784,0.743243,0.878378,0.594595,0.527027,0.081081,...,0.0,0.202703,0.067568,0.337838,1.000000,0.114865,4.071062,1.027448,148,0.000000
2,0.220779,0.493506,0.519481,0.480519,0.181818,0.597403,0.818182,0.623377,0.519481,0.077922,...,0.0,0.116883,0.025974,0.324675,1.000000,0.090909,3.845700,0.484285,154,0.000000
3,0.102273,0.511364,0.545455,0.465909,0.147727,0.659091,0.829545,0.636364,0.568182,0.045455,...,0.0,0.215909,0.045455,0.329545,1.000000,0.073864,3.822098,0.893575,176,0.000000
4,0.215190,0.455696,0.632911,0.417722,0.164557,0.658228,0.911392,0.607595,0.607595,0.025316,...,0.0,0.202532,0.037975,0.341772,1.000000,0.056962,3.727347,0.713715,158,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0.120000,0.520000,0.720000,0.440000,0.240000,0.560000,0.920000,0.280000,0.720000,0.040000,...,0.0,0.280000,0.040000,0.280000,1.000000,0.100000,3.073075,0.615696,50,0.000000
262,0.160000,0.600000,0.680000,0.800000,0.280000,0.720000,0.920000,0.560000,0.520000,0.000000,...,0.0,0.240000,0.080000,0.360000,0.880000,0.100000,5.954342,1.723439,50,0.775629
263,0.129032,0.580645,0.548387,0.645161,0.096774,0.774194,0.903226,0.709677,0.677419,0.032258,...,0.0,0.193548,0.032258,0.193548,0.903226,0.064516,3.314164,1.155196,62,0.591300
264,0.095238,0.619048,0.666667,0.619048,0.285714,0.666667,0.714286,0.571429,0.476190,0.238095,...,0.0,0.285714,0.095238,0.333333,0.952381,0.119048,3.695211,0.919168,42,0.425918


In [6]:
ep15 = vecs_trim[vecs_trim.Poem=='Ep. 15'].drop(['Author','Work','Poem'],axis=1)
ep15

,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,H4CF,H1DI,H2DI,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
14,0.275229,0.458716,0.633028,0.495413,0.155963,0.678899,0.87156,0.53211,0.559633,0.091743,...,0.0,0.174312,0.055046,0.385321,1.0,0.12844,3.004543,0.633201,218,0.0


In [35]:
# A random poem to compare to Ovid

prop3_10 = vecs_trim[vecs_trim.Poem=='Prop. 3 10'].drop(['Author','Work','Poem'],axis=1)
prop3_10

,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,H4CF,H1DI,H2DI,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
189,0.4375,0.6875,0.625,0.6875,0.375,0.75,0.9375,0.3125,0.4375,0.0625,...,0.0,0.25,0.0625,0.3125,1.0,0.1875,4.598141,0.881995,32,0.0


# A note on the Mahalanobis distance

The redoutable wikipedia has a [quick primer](https://en.wikipedia.org/wiki/Mahalanobis_distance) on the Mahalanobis distance, but the intuition is not too difficult (at least for those with some undergraduate statistics!). It is more or less like the euclidean distance, except it takes into account correlations between features. For _m_ observations in an _n_ dimensional feature space, the _covariance matrix_ is an _n_ x _n_ matrix that describes all the pairwise correlations between the features. The inverse of this matrix is then used to "correct" for those correlations. Because of the way the vectors are multiplied, it is also possible to save the product vector to see exactly which features contribute the most distance to the overall score, which is a very useful tool for interpretability. Note that in all cases below I actually measure the _squared_ M-distance. This has no effect on any comparisons, but the squared M-distance is chi-square distributed, which makes it easy to calculate a _P_-value for any distance.

In [26]:
# The pretty printing method in my mahalanobis.py was written before
# any non-percentage features were added, so this is a hack :/

def print_maha_res(samp,dist):   
    v,m,p = maha.explain(samp, dist)
    dist_cent = dist.mean(axis=0)
    print('-'*36)
    print("  M-dist %.2f,  p-value: %.4f" % (m,p))
    print("  Feat \t Score \t   Samp      Dist")
    print('-'*36)
    v=v.mean(axis=0).sort_values(ascending=False)
    for feat,score in v.iteritems():
        if feat in ('LEO','ELC','RS', 'LEN', 'PFSD'):
            print("%4.4s   %6.2f    %6.2f    %6.2f" % (feat, score, samp[feat], dist_cent[feat]))
        else:
            print("%4.4s   %6.2f    %6.2f%%   %6.2f%%" % (feat, score, samp[feat]*100, dist_cent[feat]*100))
    print('-'*36)

# Deconstructed Mahalanobis distance of _Ep._ 15 from Ovidian tendency

There are very few features that differ significantly from typical Ovidian style. The length shows up as a difference (all of the _Heroides_ are longer than most of Ovid's short elegy), as do some minor differences in the caesurae in the third and fourth feet of the hexameter, but that's about it. As for the _P_-value, there is clearly no statistical reason to reject the null hypothesis (ie no reason to reject the idea that it was written by Ovid).

In [31]:
# Nothing to see here

print_maha_res(nux_vec, ovid_dist)

------------------------------------
  M-dist 25.82,  p-value: 0.9766
  Feat 	 Score 	   Samp      Dist
------------------------------------
 LEN     4.71    182.00     79.82
P1DI     4.66     40.66%    52.00%
  RS     4.36      3.10      3.98
PFSD     3.15      0.00      0.08
H3WC     2.84      7.69%     5.36%
H4SP     2.82     62.64%    53.89%
P2SP     1.94     71.43%    61.37%
H4DI     1.60     62.64%    51.39%
P3SC     1.15     16.48%    22.07%
H4WC     1.04      1.10%     5.39%
H1SC     1.03     58.24%    49.32%
H3CF     0.92     86.81%    89.55%
 LEO     0.87      0.52      0.78
H4CF     0.67     53.85%    56.31%
H2WC     0.53     13.19%    10.13%
P4WC     0.48    100.00%    99.07%
H2SP     0.38     45.05%    52.13%
P1SC     0.38     34.07%    38.19%
P3WC     0.37     28.57%    32.27%
 ELC     0.08      0.08      0.09
P4SC     0.06      0.00%     0.20%
H1DI     0.05     60.44%    59.75%
H2SC     0.04     60.44%    58.88%
H3DI     0.04     24.18%    24.39%
P3DI     0.04     54.95%

/Users/ben/venv/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/var/folders/wt/_hvfndq56jvd5k42nrkq9fy00000gn/T/ipykernel_42939/1468742357.py:12: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for feat,score in v.iteritems():


In [53]:
# Nothing to see here

print_maha_res(nux_vec, ovid_late)

------------------------------------
  M-dist 38.68,  p-value: 0.6175
  Feat 	 Score 	   Samp      Dist
------------------------------------
H3WC     9.03      7.69%     3.96%
PFSD     5.85      0.00      0.12
 LEN     5.50    182.00     70.02
H3CF     4.60     86.81%    90.79%
H4SP     3.74     62.64%    54.48%
P1DI     3.40     40.66%    51.41%
P3SC     3.35     16.48%    24.61%
H1SC     3.06     58.24%    49.14%
P2SP     2.13     71.43%    62.04%
H4WC     2.08      1.10%     5.00%
H2WC     1.99     13.19%     9.35%
H4CF     1.81     53.85%    57.03%
  RS     1.74      3.10      3.95
H2SP     1.57     45.05%    53.81%
P2SC     1.36     72.53%    64.82%
 LEO     1.25      0.52      0.78
H1WC     1.20     16.48%    23.10%
H4DI     1.10     62.64%    52.89%
P4WC     0.63    100.00%    98.52%
H3DI     0.50     24.18%    25.73%
P3WC     0.13     28.57%    29.55%
P4SC     0.06      0.00%     0.27%
H1SP     0.06     15.38%    13.36%
P4DI     0.04      0.00%     0.07%
P2WC     0.03      4.40

/Users/ben/venv/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/var/folders/wt/_hvfndq56jvd5k42nrkq9fy00000gn/T/ipykernel_42939/1468742357.py:12: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for feat,score in v.iteritems():


In [58]:
# Nothing to see here

print_maha_res(nux_vec, ovid_early)

/Users/ben/venv/py39/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


LinAlgError: singular matrix

# The method works, in general

A random poem by Propertius is, unsurprisingly, very not-Ovidian. Here I picked Propertius 3.10 more or less at random, to demonstrate that non-Ovidian works are usually easily detectable as non-Ovidian style.

The biggest differences (after correcting for feature covariance)
- Prop 3.10 is much more spondaic in the first and second feet
  of the hexameter than the Ovidian norm
- Prop. 3.10 is less likely to have a diaeresis in H1 (which happens when the
  first foot is a disyllable)
- the poem has much more elision than is typical for Ovid
- the poem has no ictus conflicts after the caesura in the pentameter, whereas
  Ovid apparently does this one line in seven or eight (although Prop 3.5 is only 32 lines)

In [40]:
print_maha_res(prop3_10, ovid_dist)

------------------------------------
  M-dist 108.23,  p-value: 0.0000
  Feat 	 Score 	   Samp      Dist
------------------------------------
H1SP     9.95     43.75%    15.70%
H1CF     8.93     37.50%    15.30%
H3CF     8.22     93.75%    89.55%
H1DI     7.68     43.75%    59.75%
H1WC     7.47      6.25%    21.75%
H4WC     7.27     12.50%     5.39%
P3CF     7.01      0.00%    13.36%
P1CF     6.96     37.50%    26.85%
H2SP     6.75     68.75%    52.13%
H4DI     5.44     62.50%    51.39%
P1WC     5.18     25.00%    18.79%
 ELC     4.27      0.19      0.09
H3DI     3.79     12.50%    24.39%
H4CF     3.64     31.25%    56.31%
H1SC     3.47     37.50%    49.32%
P2CF     3.37     81.25%    73.65%
H4SP     3.27     68.75%    53.89%
H4SC     2.97     37.50%    68.70%
P2WC     1.98      6.25%     4.43%
P4CF     1.94      0.00%     0.76%
 LEN     1.11     32.00     79.82
P2SP     1.00     68.75%    61.37%
H3SC     0.86     93.75%    94.43%
P3SC     0.58     25.00%    22.07%
  RS     0.45      4

# Testing the accuracy

_Heroides_ 15 reads as Ovidian, and Propertius 3.10 reads as non-Ovidian, but it is worth checking the general accuracy. Here I just look quickly at the number of false positives and negatives when working at the 99% confidence level. It seems that the method is not actually 99% accurate (which is not all that surprising) but nevertheless it does a very good job. 5 of 102 non-Ovidian works might be mistaken for Ovid (about 5%), and just 14 of 164 Ovidian works are sufficiently unusual as to read as non-Ovidian (2.4%), almost all of which are later works.


In [44]:
# A quick function we can apply to the dataframe to add the M-dist
# and p-value (compared to Ovidian style) for every work in the corpus

def maha_from_ovid(row):
    
    x = pd.DataFrame(row.drop(['Author','Work','Poem'])).reset_index(drop=True).T
    x.columns = ovid_dist.columns

    x_minus_mu = x - np.mean(ovid_dist, axis=0)
    cov = np.cov(ovid_dist.values.T)
    inv_covmat = sp.linalg.inv(cov)
    left_term = np.dot(x_minus_mu, inv_covmat)

    # for the normal Mahalanobis distance we would take the dot product here
    # but instead we multiply the vectors pointwise (as in .dot) but don't add
    # up the entries. This lets us see how much each column contributes to the
    # distance. 

    v = left_term*x_minus_mu

    m = np.array(np.dot(left_term, x_minus_mu.T)[0],dtype=np.float64)
    p = 1 - sp.stats.chi2.cdf(m, len(x.columns)-1)[0]
    return pd.Series([m[0],p])

In [21]:
dist_vecs = vecs_trim.apply(maha_from_ovid, axis=1)

In [22]:
dists = vecs_trim.copy()
dists.insert(3,'OvDist',dist_vecs[0])
dists.insert(4,'pval',dist_vecs[1])

In [45]:
# false positives - non-Ovidian detected as Ovid

dists[dists.Author != 'Ovid'].sort_values(by='OvDist').query('pval > 0.01')

,Author,Work,Poem,OvDist,pval,H1SP,H2SP,H3SP,H4SP,H1CF,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
214,Propertius,Prop.,Prop. 4 11,38.302102,0.634049,0.313725,0.588235,0.627451,0.666667,0.313725,...,0.0,0.254902,0.019608,0.294118,1.000000,0.127451,4.526738,1.394520,102,0.000000
209,Propertius,Prop.,Prop. 4 6,48.673408,0.222227,0.279070,0.627907,0.720930,0.604651,0.139535,...,0.0,0.279070,0.093023,0.279070,1.000000,0.162791,4.538231,1.191339,86,0.000000
203,Propertius,Prop.,Prop. 3 24,57.081025,0.060268,0.315789,0.473684,0.578947,0.526316,0.210526,...,0.0,0.157895,0.000000,0.421053,1.000000,0.210526,4.517161,1.053250,38,0.000000
207,Propertius,Prop.,Prop. 4 4,63.732554,0.016845,0.319149,0.553191,0.595745,0.638298,0.276596,...,0.0,0.063830,0.085106,0.404255,0.978723,0.234043,3.796900,1.176762,94,0.288610
121,Tibullus,Tib.,Tib. 1 4,64.833786,0.013395,0.309524,0.452381,0.666667,0.666667,0.214286,...,0.0,0.142857,0.023810,0.214286,0.952381,0.059524,4.867841,0.867571,84,0.543462


In [46]:
# false negatives - Ovidian detected as non-Ovidian

dists[dists.Author == 'Ovid'].sort_values(by='OvDist').query('pval < 0.01')

,Author,Work,Poem,OvDist,pval,H1SP,H2SP,H3SP,H4SP,H1CF,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
248,Ovid,Pont.,Pont. 3 8,66.488886,9.407274e-03,0.166667,0.583333,0.416667,0.416667,0.000000,...,0.000000,0.416667,0.083333,0.166667,1.000000,0.000000,3.800638,0.341329,24,0.000000
48,Ovid,Tr.,Tr. 4 2,66.509220,9.365903e-03,0.216216,0.540541,0.702703,0.702703,0.189189,...,0.000000,0.162162,0.081081,0.243243,0.972973,0.135135,3.803756,0.986833,74,0.000000
32,Ovid,Tr.,Tr. 2 1,66.778613,8.833370e-03,0.138408,0.570934,0.619377,0.525952,0.138408,...,0.000000,0.217993,0.044983,0.273356,0.979239,0.086505,3.872211,0.845058,578,0.386243
81,Ovid,Am.,Am. 1 11,67.388686,7.728845e-03,0.142857,0.428571,0.642857,0.571429,0.214286,...,0.000000,0.214286,0.000000,0.500000,1.000000,0.285714,4.260925,0.588626,28,0.000000
40,Ovid,Tr.,Tr. 3 8,70.400461,3.918242e-03,0.095238,0.523810,0.666667,0.666667,0.095238,...,0.000000,0.190476,0.047619,0.380952,1.000000,0.071429,2.727267,0.176037,42,0.000000
92,Ovid,Am.,Am. 2 8,72.913442,2.169994e-03,0.214286,0.642857,0.428571,0.785714,0.214286,...,0.000000,0.142857,0.000000,0.428571,1.000000,0.107143,4.158458,1.479839,28,0.000000
262,Ovid,Pont.,Pont. 4 13,77.930369,6.278599e-04,0.160000,0.600000,0.680000,0.800000,0.280000,...,0.000000,0.240000,0.080000,0.360000,0.880000,0.100000,5.954342,1.723439,50,0.775629
246,Ovid,Pont.,Pont. 3 6,83.136986,1.604331e-04,0.166667,0.433333,0.666667,0.533333,0.233333,...,0.033333,0.200000,0.033333,0.333333,0.966667,0.083333,4.160358,1.065353,60,0.179505
69,Ovid,Tr.,Tr. 5 13,84.153643,1.218745e-04,0.000000,0.588235,0.588235,0.411765,0.058824,...,0.058824,0.294118,0.000000,0.176471,0.941176,0.029412,3.437513,0.824276,34,0.000000
24,Ovid,Tr.,Tr. 1 4,85.246547,9.042909e-05,0.071429,0.285714,0.857143,0.571429,0.071429,...,0.000000,0.071429,0.214286,0.285714,0.928571,0.035714,3.982128,1.291192,28,0.515079


In [29]:
dists[dists.Author == 'Ovid'].shape[0]

164

In [30]:
dists[dists.Author != 'Ovid'].shape[0]

102

# All of the _Heroides_ sorted by M-distance (larger scores less like 'typical' Ovidian style

Note that even the most different are nowhere near different enough to be in statistical doubt. Of course this doesn't prove that they are Ovidian, but it does show that they conform superbly well with every feature that was measured.

In [210]:
dists[dists.Work == 'Ep.'].sort_values(by='OvDist')

,Author,Work,Poem,OvDist,pval,H1SP,H2SP,H3SP,H4SP,H1CF,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
4,Ovid,Ep.,Ep. 5,13.390092,0.999992,0.215190,0.455696,0.632911,0.417722,0.164557,...,0.000000,0.202532,0.037975,0.341772,1.000000,0.056962,3.727347,0.713715,158,0.000000
3,Ovid,Ep.,Ep. 4,17.860155,0.999602,0.102273,0.511364,0.545455,0.465909,0.147727,...,0.000000,0.215909,0.045455,0.329545,1.000000,0.073864,3.822098,0.893575,176,0.000000
5,Ovid,Ep.,Ep. 6,19.191914,0.999029,0.168675,0.445783,0.554217,0.445783,0.180723,...,0.000000,0.228916,0.036145,0.325301,1.000000,0.090361,4.489172,0.689778,166,0.000000
12,Ovid,Ep.,Ep. 13,19.308738,0.998955,0.172840,0.493827,0.543210,0.518519,0.160494,...,0.000000,0.259259,0.049383,0.345679,1.000000,0.098765,4.916439,0.755515,162,0.000000
18,Ovid,Ep.,Ep. 19,19.625232,0.998730,0.104762,0.514286,0.647619,0.514286,0.171429,...,0.000000,0.161905,0.028571,0.342857,0.990476,0.071429,4.116780,0.799599,210,0.194248
20,Ovid,Ep.,Ep. 21,22.911907,0.992792,0.211382,0.512195,0.682927,0.552846,0.219512,...,0.000000,0.243902,0.024390,0.300813,1.000000,0.065041,4.251277,0.895832,246,0.000000
10,Ovid,Ep.,Ep. 11,23.645690,0.990019,0.078125,0.562500,0.500000,0.562500,0.140625,...,0.000000,0.140625,0.046875,0.390625,1.000000,0.109375,4.608592,0.868259,128,0.000000
19,Ovid,Ep.,Ep. 20,23.775092,0.989451,0.159664,0.529412,0.672269,0.470588,0.201681,...,0.008403,0.142857,0.025210,0.302521,0.991597,0.050420,3.802721,0.516711,238,0.000000
2,Ovid,Ep.,Ep. 3,24.904209,0.983319,0.220779,0.493506,0.519481,0.480519,0.181818,...,0.000000,0.116883,0.025974,0.324675,1.000000,0.090909,3.845700,0.484285,154,0.000000
17,Ovid,Ep.,Ep. 18,26.441128,0.970857,0.128440,0.633028,0.568807,0.412844,0.211009,...,0.009174,0.220183,0.036697,0.174312,0.990826,0.064220,3.807675,0.724245,218,0.000000


# A quick note on Am. 3.5

_Amores_ 3.5 is not accepted by some editors. According to the M-distance, there is insufficient statistical reason to reject it on the grounds of poetic style. This is included as a fairly quick aside, but it may be of interest to some.

In [31]:
dists[dists.Poem=='Am. 3 5']

,Author,Work,Poem,OvDist,pval,H1SP,H2SP,H3SP,H4SP,H1CF,...,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,LEN,PFSD
108,Ovid,Am.,Am. 3 5,45.208894,0.339455,0.130435,0.521739,0.652174,0.521739,0.086957,...,0.0,0.304348,0.0,0.304348,1.0,0.086957,4.548982,0.481772,46,0.0
